<a href="https://colab.research.google.com/github/2ndt2nd/Final-Project-IAT-360/blob/main/Final_Project_Jacques_Seth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><h1> <b> IAT 360 Final Project <b> </h1></center>

This is the final project for IAT 360
Group Members:
    - Seth van der Zweep (301461888)
    - Jacques Davidson Widodo (301443163)


<h2>Importing required Libraries<h2>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import seaborn as sns
import glob
import xml.etree.ElementTree as ET
from PIL import Image
import os
import shutil

2024-11-27 15:57:23.192844: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-27 15:57:23.227621: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732751843.268427     379 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732751843.280894     379 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-27 15:57:23.320147: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

<strong>Import Ultralytics<strong>


In [1]:
import ultralytics
ultralytics.checks()

from ultralytics import YOLO

model = YOLO('best.pt')

Ultralytics 8.3.38 🚀 Python-3.12.7 torch-2.5.1+cu124 CPU (11th Gen Intel Core(TM) i5-1135G7 2.40GHz)
Setup complete ✅ (8 CPUs, 19.3 GB RAM, 0.0/9.7 GB disk)


<h1>Process Feed</h1>
Running the testing to see if webcam works

In [2]:
import cv2
import time

cap = cv2.VideoCapture(0)  # Try different indices like 0, 1, 2
if not cap.isOpened():
    print("Error: Could not open webcam.")
else:
    print("Webcam opened successfully!")
    
cap.release()


Webcam opened successfully!


<h2>Main Code</h2>
Below is the code that contains the main process.

Right now it captures the webcam, and then annotates the frames with the classes.

    [100, 50, 200, 150, 0.85, 3],  # Detected object 1

    [100, 50, 200, 150] → Bounding box: top-left (x_min, y_min) and bottom-right (x_max, y_max).
    0.85 → Confidence score.
    3 → Class ID (e.g., "car" if class 3 maps to "car").

    From results below we could get the class and see if it would trigger an input

In [ ]:
def process_video_feed(model):
    cap = cv2.VideoCapture(0, cv2.CAP_V4L2)  # Adjust backend for your OS
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 8)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 3.5)
    cap.set(cv2.CAP_PROP_FPS, 10)  # Adjust FPS
    
    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return

    while True:
        start_time = time.time()
        ret, frame = cap.read()  # Capture frame
        if not ret:
            print("Error: Failed to grab frame.")
            break

        frame = cv2.resize(frame, (640, 480))

        results = model.predict(frame, conf=0.75)  # YOLO inference
        rendered_frame = results[0].plot()
        cv2.imshow("Webcam Feed", rendered_frame)

        print(f"Frame processing time: {time.time() - start_time:.3f} seconds")
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


<h1>When you're ready to run</h1>
Just run the code below!

In [ ]:
# Start processing the webcam feed
process_video_feed(model)

<h3>Release Cam</h3>
In case you need to release the cam

In [31]:
cap.release()
cv2.destroyAllWindows()